__GERMANY HEART ATTACK ANALYSIS__

In [1]:
import mysql.connector
import pandas as pd

In [ ]:
# Database Configuration (Use your credentials to connect with MySQL Database)
DB_CONFIG = {
    "host": "#hostname",
    "user": "#username",
    "password": "#your_password",  # Ensure this is secure in production!
    "database": "#database_name"
}

# Create a connection
try:
    connection = mysql.connector.connect(**DB_CONFIG)
    print("Connection Successful")
except mysql.connector.Error as err:
    print(f"Connection Error: {err}")

In [ ]:
def fetch_query_results(query):
    """Executes a SQL query and returns a Pandas DataFrame."""
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            return pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    except mysql.connector.Error as err:
        print(f"Error executing query: {err}")
        return None

__1. Find the total number of heart attack incidences for each age group in Germany.__

In [ ]:
query1 = """
SELECT age_group, COUNT(*) AS number_of_heart_attack_incidence
FROM heart_attack_germany
WHERE heart_attack_incidence = 1
GROUP BY age_group;
"""
res1 = fetch_query_results(query1)
res1


__2. Calculate the average BMI for each gender across all states.__

In [ ]:
query2 = """
SELECT state, gender, ROUND(AVG(bmi),4) AS avg_bmi
FROM heart_attack_germany
GROUP BY state, gender;;
"""
res2 = fetch_query_results(query2)
res2

__3. List the top 5 states with the highest heart attack incidence rates among youth.__

In [ ]:
query3 = """
SELECT state, COUNT(*) AS number_of_heart_attack_incidence
FROM heart_attack_germany
WHERE heart_attack_incidence = 1
GROUP BY state
ORDER BY number_of_heart_attack_incidence DESC
LIMIT 5;
"""
res3 = fetch_query_results(query3)
res3

__4. Count the number of heart attack incidences in urban areas compared to rural areas.__

In [ ]:
query4 = """
SELECT urban_rural, COUNT(*) AS number_of_heart_attack_incidence
FROM heart_attack_germany
WHERE heart_attack_incidence = 1
GROUP BY urban_rural;		
"""
res4 = fetch_query_results(query4)
res4

__5.	Find the average heart attack incidence by socioeconomic status.__

In [ ]:
query5 = """ 
SELECT socioeconomic_status, AVG(heart_attack_incidence) AS number_of_heart_attack_incidence
FROM heart_attack_germany
GROUP BY socioeconomic_status;
"""
res5 = fetch_query_results(query5)
res5

__6.	Identify the year with the highest heart attack incidences for adults.__

In [ ]:
query6 = """ 
SELECT years, COUNT(*) AS number_of_heart_attack_incidence
FROM heart_attack_germany
WHERE heart_attack_incidence = 1
GROUP BY years
ORDER BY number_of_heart_attack_incidence DESC
LIMIT 1; # year is 2017 with 4686 cases of heartattack 
"""
res6 = fetch_query_results(query6)
res6

__7.	Compare the heart attack incidence rates between youth and adults by region.__

In [ ]:
query7 = """ 
SELECT urban_rural AS region, age_group, COUNT(*) AS number_of_heart_attack_incidence
FROM heart_attack_germany
WHERE heart_attack_incidence = 1
GROUP BY urban_rural, age_group;
"""
res7 = fetch_query_results(query7)
res7

__8.	Calculate the percentage of smokers who have had heart attacks across all age groups.__

In [ ]:
query8 = """ 
SELECT 
ROUND(((
            SELECT COUNT(heart_attack_incidence) 
            FROM heart_attack_germany 
            WHERE heart_attack_incidence = 1 AND smoking_status != 'Non-Smoker'
        )/COUNT(*) *100),2) AS percentage_of_smokers_who_have_had_heart_attacks
FROM heart_attack_germany;
"""
res8 = fetch_query_results(query8)
res8

__9.	Find the average physical activity level for states with below-average heart attack rates.__

In [ ]:
query9 = """ 
SELECT state AS states_with_below_avg_heart_attack_rates , ROUND(AVG(region_heart_attack_rate),4) AS avg_heart_attack_rates
FROM heart_attack_germany
GROUP BY state
HAVING (SELECT AVG(region_heart_attack_rate) FROM heart_attack_germany) <  avg_heart_attack_rates
ORDER BY avg_heart_attack_rates DESC;
"""
res9 = fetch_query_results(query9)
res9

__10.	List the years in which alcohol consumption was above the national average.__

In [ ]:
query10 = """ 
SELECT years, ROUND(AVG(alcohol_cosumption),4) AS avg_acohol_consumption_above_national_average
FROM heart_attack_germany
GROUP BY years
HAVING (SELECT ROUND(AVG(alcohol_cosumption),4) AS avg_national_acohol_consumption FROM heart_attack_germany) < avg_acohol_consumption_above_national_average
ORDER BY avg_acohol_consumption_above_national_average DESC;
"""
res10 = fetch_query_results(query10)
res10

__11.	Group the data by gender and calculate the median heart attack incidence.__

In [ ]:
query11 = """ 
SELECT gender, AVG(heart_attack_incidence) AS median_value
FROM 
	(
		SELECT gender, heart_attack_incidence,
		ROW_NUMBER() OVER (PARTITION BY gender ORDER BY heart_attack_incidence) AS row_num,
		COUNT(*) OVER (PARTITION BY gender) AS total_count
		FROM heart_attack_germany
	) as RankData
WHERE (total_count % 2 = 1 AND row_num = (total_count + 1)/2)
OR (total_count % 2 = 0 AND (row_num = (total_count/2) OR row_num = (total_count + 1)/2))
GROUP BY gender;
"""
res11 = fetch_query_results(query11)
res11

__12.	Find the maximum and minimum heart attack incidences for each state and year.__

In [ ]:
query12 = """ 
SELECT state, years,
    MAX(heart_attack_incidence) AS max_incidence,
    MIN(heart_attack_incidence) AS min_incidence
FROM heart_attack_germany
GROUP BY state, years
"""
res12 = fetch_query_results(query12)
res12

__13.	Calculate the average stress level for states with the top 10 highest air pollution indices.__

In [ ]:
query13 = """ 
WITH Top10AirPollution AS 
        (
            SELECT state
            FROM heart_attack_germany
            ORDER BY air_pollution_index DESC
            LIMIT 10
        )

SELECT state,
    AVG(CASE
            WHEN stress_level = "High" THEN 3
            WHEN stress_level = "Moderate" THEN 2
            WHEN stress_level = "Low" THEN 1
        END
    ) AS avg_stress_level
FROM heart_attack_germany
WHERE state IN (SELECT state FROM Top10AirPollution)
GROUP BY state;
"""
res13 = fetch_query_results(query13)
res13

__14.	Group the data by education level and find the average cholesterol level for each group.__

In [ ]:
query14 = """ 
SELECT education_level, ROUND(AVG(cholestrol_level),4) AS avg_cholestrol_level
FROM heart_attack_germany
GROUP BY education_level;
"""
res14 = fetch_query_results(query14)
res14

__15.	Calculate the average heart attack rate in regions with above-average healthcare access.__

In [ ]:
# taken urban/rural as a region. in case of depth analysis state also can be taken as region. 
query15 = """ 
WITH avg_healthcare_access AS
                            (
                                SELECT urban_rural AS region,
                                    AVG(CASE 
                                            WHEN healthcare_access = 'Easy' THEN 1
                                            WHEN healthcare_access = 'Moderate' THEN 2
                                            WHEN healthcare_access = 'Hard' THEN 3
                                            ELSE NULL
                                        END) AS avg_healthcare_access
                                FROM heart_attack_germany
                                GROUP BY urban_rural                                        
                            )

SELECT a.region, ROUND(AVG(h.region_heart_attack_rate),4) AS avg_heart_attack_rate_in_regions
FROM avg_healthcare_access a
JOIN heart_attack_germany h
ON a.region = h.urban_rural
WHERE 
    CASE 
        WHEN healthcare_access = 'Easy' THEN 1
        WHEN healthcare_access = 'Moderate' THEN 2
        WHEN healthcare_access = 'Hard' THEN 3
        ELSE NULL
    END > a.avg_healthcare_access # numbers will change if this condition is not applied
GROUP BY a.region;
"""
res15 = fetch_query_results(query15)
res15

__16.	Compare the total heart attack incidence rates for urban vs. rural areas grouped by socioeconomic status.__

In [ ]:
query16 = """ 
SELECT 
    socioeconomic_status AS "Socioeconomic Status",
    SUM(CASE WHEN urban_rural = 'Urban' THEN heart_attack_incidence ELSE 0 END) AS "Total Heart Attack Incidence (Urban)",
    SUM(CASE WHEN urban_rural = 'Rural' THEN heart_attack_incidence ELSE 0 END) AS "Total Heart Attack Incidence (Rural)"
FROM heart_attack_germany
GROUP BY socioeconomic_status;
"""
res16 = fetch_query_results(query16)
res16

__17.	Find the most common diet quality rating among youth with heart attacks.__

In [ ]:
query17 = """ 
SELECT diet_quality, COUNT(diet_quality) AS diet_quality_rating
FROM heart_attack_germany
WHERE age_group = 'Youth' AND heart_attack_incidence > 0
GROUP BY diet_quality
ORDER BY diet_quality_rating DESC
LIMIT 1;
"""
res17 = fetch_query_results(query17)
res17

__18.	Identify the states where the heart attack incidence rate increased year-over-year for three consecutive years.__

In [ ]:
query18 = """ 
WITH YearlyIncidence AS 
        (
            SELECT state, years,
                SUM(heart_attack_incidence) AS yearly_incidences
            FROM heart_attack_germany
            GROUP BY state,years
        ),
ConsecutiveIncrease AS
		(
			SELECT state,years,yearly_incidences,
				LAG(yearly_incidences) OVER(PARTITION BY state ORDER BY years) AS prev_yrs,
				LAG(yearly_incidences,2) OVER(PARTITION BY state ORDER BY years) AS two_yrs_ago
			FROM YearlyIncidence
        )

SELECT state,years,yearly_incidences,prev_yrs,two_yrs_ago
FROM ConsecutiveIncrease
WHERE yearly_incidences > prev_yrs AND prev_yrs > two_yrs_ago;
"""
res18 = fetch_query_results(query18)
res18

__19.	Calculate the average physical activity level grouped by employment status.__

In [ ]:
query19 = """ 
SELECT 
    employment_status, 
    AVG(CASE 
            WHEN physical_activity_level = 'low' THEN 1
            WHEN physical_activity_level = 'moderate' THEN 2
            WHEN physical_activity_level = 'high' THEN 3
            ELSE NULL
        END) AS avg_physical_activity_score
FROM heart_attack_germany
GROUP BY employment_status
ORDER BY employment_status;
"""
res19 = fetch_query_results(query19)
res19

__20.	Using a subquery, find the states where the average BMI is above the national average.__

In [ ]:
query20 = """ 
SELECT 
    state, 
    ROUND(AVG(bmi),4) AS avg_bmi
FROM heart_attack_germany
GROUP BY state
HAVING AVG(bmi) > (SELECT AVG(bmi) FROM heart_attack_germany)
ORDER BY avg_bmi DESC;
"""
res20 = fetch_query_results(query20)
res20

__21.	Find the states where the youth heart attack rate is higher than the adult rate using a self-join.__

In [ ]:
query21 = """ 
WITH AgeData AS 
        (
            SELECT state, age_group,
                AVG(heart_attack_incidence) AS avg_heart_attack_rate
            FROM heart_attack_germany
            WHERE age_group IN ("Youth", "Adult")
            GROUP BY state, age_group
        )
SELECT 
    y.state,
    y.avg_heart_attack_rate AS youth_rate,
    a.avg_heart_attack_rate AS adult_rate
FROM AgeData AS y
JOIN AgeData AS a
ON y.state = a.state AND y.age_group = "Youth" AND a.age_group = "Adult"
WHERE y.avg_heart_attack_rate > a.avg_heart_attack_rate;
"""
res21 = fetch_query_results(query21)
res21

__22.	Identify the regions with heart attack rates above the average for their socioeconomic status using a correlated subquery.__

In [ ]:
query22 = """ 
SELECT state, socioeconomic_status, AVG(heart_attack_incidence) AS avg_heart_attack_incidence
FROM heart_attack_germany AS t1
GROUP BY state, socioeconomic_status
HAVING AVG(heart_attack_incidence) > (
    SELECT AVG(heart_attack_incidence)
    FROM heart_attack_germany AS t2
    WHERE t1.socioeconomic_status = t2.socioeconomic_status
);
"""
res22 = fetch_query_results(query22)
res22

__23.	Write a query to find the states with the highest incidence rate for smokers and compare it to nonsmokers using a join.__

In [ ]:
query23 = """
WITH smoker_inc AS 
        (
            SELECT state,AVG(heart_attack_incidence) AS avg_incidence
            FROM heart_attack_germany
            WHERE smoking_status IN ('Smoker','Former Smoker')
            GROUP BY state
        ),
nonsmoker_inc AS
        (
            SELECT state,AVG(heart_attack_incidence) AS avg_incidence
            FROM heart_attack_germany
            WHERE smoking_status = 'Non-Smoker'
            GROUP BY state
        )
SELECT sm.state, sm.avg_incidence AS SmokerIncidence, nsm.avg_incidence AS NonSmokerIncidence
FROM smoker_inc AS sm
JOIN nonsmoker_inc AS nsm
ON sm.state = nsm.state
ORDER BY SmokerIncidence DESC;
"""
res23 = fetch_query_results(query23)
res23

__24.	Use a subquery to find states with higher-than-average stress levels but lower-than-average heart attack incidences.__

In [ ]:
query24 = """ 
SELECT state, AVG(heart_attack_incidence) AS avg_heart_attack_incidences,
	AVG(CASE
            WHEN stress_level = "High" THEN 3
            WHEN stress_level = "Moderate" THEN 2
            WHEN stress_level = "Low" THEN 1
        END
		) AS avg_stress_level
FROM heart_attack_germany
GROUP BY state
HAVING avg_heart_attack_incidences > (SELECT AVG(heart_attack_incidence) FROM heart_attack_germany)
	AND avg_stress_level > (SELECT AVG(CASE
											WHEN stress_level = "High" THEN 3
											WHEN stress_level = "Moderate" THEN 2
											WHEN stress_level = "Low" THEN 1
										END
										) AS avg_stress_level
							FROM heart_attack_germany);
"""
res24 = fetch_query_results(query24)
res24

__25.	Calculate the yearly percentage change in heart attack rates for each state.__

In [ ]:
query25 = """ 
WITH YearlyRate AS 
        (
            SELECT state,years,
                SUM(heart_attack_incidence) AS total_incidences
            FROM heart_attack_germany
            GROUP BY state,years
        ),
PrevYearRate AS
        (
            SELECT state,years, total_incidences,
                LAG(total_incidences) OVER (PARTITION BY state ORDER BY years) AS prev_year_incidence
            FROM YearlyRate
        )
SELECT state,years,
    ((total_incidences - prev_year_incidence)/prev_year_incidence)*100 AS YoY_percentage
FROM PrevYearRate
WHERE prev_year_incidence IS NOT NULL;
"""
res25 = fetch_query_results(query25)
res25

__26.	Rank the states by heart attack incidence rate in adults, partitioned by year.__

In [ ]:
query26 = """ 
WITH RankedData As
        (
            SELECT state,years,
                RANK() OVER (PARTITION BY years ORDER BY SUM(heart_attack_incidence) DESC) AS rank_by_incidence
            FROM heart_attack_germany
            WHERE age_group = "Adult"
            GROUP BY state,years
        )
SELECT state AS States,
    MAX(CASE WHEN years = 2015 THEN rank_by_incidence END) AS "2015 Rank", 
    MAX(CASE WHEN years = 2016 THEN rank_by_incidence END) AS "2016 Rank", 
    MAX(CASE WHEN years = 2017 THEN rank_by_incidence END) AS "2017 Rank", 
    MAX(CASE WHEN years = 2018 THEN rank_by_incidence END) AS "2018 Rank", 
    MAX(CASE WHEN years = 2019 THEN rank_by_incidence END) AS "2019 Rank", 
    MAX(CASE WHEN years = 2020 THEN rank_by_incidence END) AS "2020 Rank", 
    MAX(CASE WHEN years = 2021 THEN rank_by_incidence END) AS "2021 Rank", 
    MAX(CASE WHEN years = 2022 THEN rank_by_incidence END) AS "2022 Rank",
    MAX(CASE WHEN years = 2023 THEN rank_by_incidence END) AS "2023 Rank"
FROM RankedData
GROUP BY state
ORDER BY state;
"""
res26 = fetch_query_results(query26)
res26

__27.	Calculate the running total of heart attack incidences for youth in Germany, partitioned by year.__

In [ ]:
query27 = """ 
SELECT years, state,heart_attack_incidence,
       SUM(heart_attack_incidence) OVER (PARTITION BY years ORDER BY state ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS Running_Total
FROM heart_attack_germany
WHERE age_group = 'Youth';
"""
res27 = fetch_query_results(query27)
res27

__28.	Find the cumulative average cholesterol level for each state and year.__

In [ ]:
query28 = """ 
SELECT state, years, cholestrol_level, 
    AVG(cholestrol_level) OVER (PARTITION BY state ORDER BY years ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS Cumm_cholestrol_avg
FROM heart_attack_germany;
"""
res28 = fetch_query_results(query28)
res28

__29.	Use a window function to identify the top 3 states with the highest youth heart attack rates each year.__

In [ ]:
query29 = """ 
WITH TotalIncidence AS
		(
			SELECT state, years,
				SUM(heart_attack_incidence) AS total_incidence
			FROM heart_attack_germany
            WHERE age_group = "Youth" 
            GROUP BY state, years
        ),
RankedData AS 
		(
			SELECT state, years, total_incidence,
				RANK() OVER (PARTITION BY years ORDER BY total_incidence DESC) AS rank_by_incidence
			FROM TotalIncidence
        )
SELECT years, state, total_incidence, rank_by_incidence
FROM RankedData
WHERE rank_by_incidence <= 3;
"""
res29 = fetch_query_results(query29)
res29

__30.	Calculate the difference in heart attack rates between urban and rural areas for each state using a lag function.__

In [ ]:
query30 = """
SELECT state,
	MAX(CASE WHEN urban_rural = "Urban" THEN heart_attack_rate END) AS urban_heart_attack_rate,
	MAX(CASE WHEN urban_rural = "Rural" THEN heart_attack_rate END) AS rural_heart_attack_rate,
	(MAX(CASE WHEN urban_rural = "Urban" THEN heart_attack_rate END) - 
	MAX(CASE WHEN urban_rural = "Rural" THEN heart_attack_rate END)) AS rate_difference
FROM
	(
		SELECT state, urban_rural,
			AVG(heart_attack_incidence) AS heart_attack_rate
		FROM heart_attack_germany
		GROUP BY state, urban_rural
	) AS subquery
GROUP BY state
ORDER BY rate_difference DESC;
"""
res30 = fetch_query_results(query30)
res30

__31.	Find the correlation between air pollution index and heart attack incidence rates.__

In [ ]:
query31 = """
SELECT 
    (SUM((air_pollution_index - avg_x) * (heart_attack_incidence - avg_y)) / COUNT(*)) /
    (SQRT(SUM(POW(air_pollution_index - avg_x, 2)) / COUNT(*)) *
     SQRT(SUM(POW(heart_attack_incidence - avg_y, 2)) / COUNT(*))) AS Correlation_btw_API_and_HAI
FROM 
    (
        SELECT 
            air_pollution_index,
            heart_attack_incidence,
            (SELECT AVG(air_pollution_index) FROM heart_attack_germany) AS avg_x,
            (SELECT AVG(heart_attack_incidence) FROM heart_attack_germany) AS avg_y
        FROM 
            heart_attack_germany
    ) AS subquery;
"""

res31 = fetch_query_results(query31)
res31

__32.	Identify the regions with a consistent increase in heart attack rates across all socioeconomic levels.__

In [ ]:
query32 = """ 
WITH RegionRates AS
        (
            SELECT state, socioeconomic_status, years,
                SUM(heart_attack_incidence) AS total_incidences
            FROM heart_attack_germany
            GROUP BY state, socioeconomic_status, years
        ),
PrevYearIncidences AS
        ( 
            SELECT state, socioeconomic_status, years, total_incidences, 
                LAG(total_incidences) OVER (PARTITION BY state, socioeconomic_status ORDER BY years) AS previous_yr_incidences
            FROM RegionRates
        )
SELECT state, socioeconomic_status,
    COUNT(years) AS YoY_Increase_count
FROM PrevYearIncidences
WHERE total_incidences > previous_yr_incidences
GROUP BY state, socioeconomic_status
HAVING COUNT(years) > 1;
"""
res32 = fetch_query_results(query32)
res32

__33.	Analyze the effect of diabetes on heart attack incidences for different age groups using a group-by analysis.__

In [ ]:
query33 = """ 
SELECT age_group,
	AVG(CASE WHEN diabetes = 0 THEN heart_attack_incidence END) AS non_diabetic,
    AVG(CASE WHEN diabetes = 1 THEN heart_attack_incidence END) AS diabetic
FROM heart_attack_germany
GROUP BY age_group;
"""
res33 = fetch_query_results(query33)
res33

__34.	Calculate the year-over-year growth in heart attack incidences for youth in Germany.__

In [ ]:
query34 = """ 
WITH YoY_Growth AS
        (
            SELECT years,
                SUM(heart_attack_incidence) AS total_incidences
            FROM heart_attack_germany
            WHERE age_group = "Youth"
            GROUP BY years
        )
SELECT years,total_incidences,
    (total_incidences - LAG(total_incidences) OVER (ORDER BY years)) / LAG(total_incidences) OVER (ORDER BY years) * 100 AS Growth_Percentage,
    SUM(total_incidences) OVER (ORDER BY years) AS YoY_Growth_Incidents
FROM YoY_Growth;
"""
res34 = fetch_query_results(query34)
res34

__35.	Determine if smoking status or alcohol consumption has a stronger correlation with heart attack incidences.__

In [ ]:
query35 = """ 
WITH AlcoholCorrelation AS (
    SELECT 
        (SUM((alcohol_cosumption - avg_x) * (heart_attack_incidence - avg_y)) / COUNT(*)) /
        (SQRT(SUM(POW(alcohol_cosumption - avg_x, 2)) / COUNT(*)) *
         SQRT(SUM(POW(heart_attack_incidence - avg_y, 2)) / COUNT(*))) AS correlation_alcohol
    FROM (
        SELECT 
            alcohol_cosumption,
            heart_attack_incidence,
            (SELECT AVG(alcohol_cosumption) FROM heart_attack_germany) AS avg_x,
            (SELECT AVG(heart_attack_incidence) FROM heart_attack_germany) AS avg_y
        FROM 
            heart_attack_germany
    ) AS subquery
),
SmokingCorrelation AS (
    SELECT 
        (SUM((smoking_code - avg_x) * (heart_attack_incidence - avg_y)) / COUNT(*)) /
        (SQRT(SUM(POW(smoking_code - avg_x, 2)) / COUNT(*)) *
         SQRT(SUM(POW(heart_attack_incidence - avg_y, 2)) / COUNT(*))) AS correlation_smoking
    FROM (
        SELECT 
            CASE 
                WHEN smoking_status = 'Smoker' THEN 3
                WHEN smoking_status = 'Former Smoker' THEN 2
                WHEN smoking_status = 'Non-Smoker' THEN 1
            END AS smoking_code,
            heart_attack_incidence,
            (SELECT AVG(CASE 
                            WHEN smoking_status = 'Smoker' THEN 3
                            WHEN smoking_status = 'Former Smoker' THEN 2
                            WHEN smoking_status = 'Non-Smoker' THEN 1
                        END)
             FROM heart_attack_germany) AS avg_x,
            (SELECT AVG(heart_attack_incidence) FROM heart_attack_germany) AS avg_y
        FROM 
            heart_attack_germany
    ) AS subquery
)
SELECT 
    AlcoholCorrelation.correlation_alcohol AS "Correlation between alcohol consumption and heart attack incidence",
    SmokingCorrelation.correlation_smoking AS "Correlation between smoking habit and heart attack incidence"
FROM 
    AlcoholCorrelation, SmokingCorrelation;
"""
res35 = fetch_query_results(query35)
res35

__36.	Analyze the impact of education level on physical activity levels and its effect on heart attack rates.__

In [ ]:
query36 = """ 
SELECT education_level,
	AVG(CASE WHEN physical_activity_level = 'High' THEN heart_attack_incidence END) AS high_phy_act,
    AVG(CASE WHEN physical_activity_level = 'Moderate' THEN heart_attack_incidence END) AS moderate_phy_act,
    AVG(CASE WHEN physical_activity_level = 'Low' THEN heart_attack_incidence END) AS low_phy_act
FROM heart_attack_germany
GROUP BY education_level;
"""
res36 = fetch_query_results(query36)
res36

__37.	Identify the states where family history has the highest influence on heart attack rates, controlling for age group and gender.__

In [ ]:
query37 = """ 
WITH FamilyHistoryRates AS (
    SELECT state, age_group, gender, family_history,
           AVG(heart_attack_incidence) AS avg_incidence
    FROM heart_attack_germany
    GROUP BY state, age_group, gender, family_history
),
DifferenceByFamilyHistory AS (
    SELECT state, age_group, gender,
           MAX(CASE WHEN family_history = 1 THEN avg_incidence ELSE 0 END) -
           MAX(CASE WHEN family_history = 0 THEN avg_incidence ELSE 0 END) AS influence_level
    FROM FamilyHistoryRates
    GROUP BY state, age_group, gender
)
SELECT state, age_group, gender, influence_level
FROM DifferenceByFamilyHistory
ORDER BY influence_level DESC
LIMIT 10;
"""
res37 = fetch_query_results(query37)
res37

__38.	Use a CTE (Common Table Expression) to find the average diet quality and its relationship to heart attack rates in adults.__

In [ ]:
query38 = """ 
WITH DietQualityRates AS (
    SELECT diet_quality, 
           AVG(heart_attack_incidence) AS avg_incidence,
           COUNT(*) AS total_records
    FROM heart_attack_germany
    WHERE age_group = 'Adult'
    GROUP BY diet_quality
)
SELECT diet_quality, avg_incidence, total_records,
       CASE 
           WHEN avg_incidence > (SELECT AVG(heart_attack_incidence) FROM heart_attack_germany WHERE age_group = 'Adult') 
           THEN 'High Risk'
           ELSE 'Low Risk'
       END AS risk_level
FROM DietQualityRates;
"""
res38 = fetch_query_results(query38)
res38

__39.	Analyze the relationship between hypertension and cholesterol levels and their combined effect on heart attack incidence rates.__

In [ ]:
query39 = """ 
SELECT hypertension, 
       ROUND(AVG(cholestrol_level),4) AS avg_cholesterol,
       ROUND(AVG(heart_attack_incidence),4) AS avg_heart_attack_rate
FROM heart_attack_germany
GROUP BY hypertension
ORDER BY avg_heart_attack_rate DESC;
"""
res39 = fetch_query_results(query39)
res39

__40.	Write a query to segment the population into high-risk and low-risk groups for heart attacks based on stress level, BMI, and healthcare access.__

In [ ]:
query40 = """ 
WITH RiskSegmentation AS (
    SELECT stress_level, bmi, healthcare_access, 
           CASE 
               WHEN stress_level = 'High' AND bmi > 30 AND healthcare_access = 'Hard' THEN 'High Risk'
               WHEN stress_level = 'Moderate' AND bmi BETWEEN 25 AND 30 AND healthcare_access = 'Moderate' THEN 'Moderate Risk'
               ELSE 'Low Risk'
           END AS risk_category
    FROM heart_attack_germany
)
SELECT risk_category, COUNT(*) AS population_count
FROM RiskSegmentation
GROUP BY risk_category
ORDER BY population_count DESC;
"""
res40 = fetch_query_results(query40)
res40

In [ ]:
connection.close()
print("Connection Closed")